In [4]:
import pandas as pd
import numpy as np
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [15]:
n = 13
dongs = ['일원동', '논현동', '역삼동', '세곡동', '압구정동', '대치동', '신사동', '삼성동', '청담동',
       '도곡동', '수서동', '자곡동', '개포동', '율현동']
data = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_20230331\소상공인시장진흥공단_상가(상권)정보_서울_202303.csv')
data = data.loc[data['상권업종대분류명'] == '음식']  
data = data.loc[data['시군구명'] == '강남구']
data = data[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '법정동명']]
data = data.loc[data['법정동명'] == dongs[n]]

data.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              ]
data

C:\Users\user\AppData\Local\Temp\ipykernel_14064\1009999047.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_20230331\소상공인시장진흥공단_상가(상권)정보_서울_202303.csv')


,name,cate_1,cate_2,cate_3,dong
6077,릴리펏수서점,비알코올,카페,커피 전문점,율현동
25106,그라운드오리진,비알코올,카페,커피 전문점,율현동
30878,깡미네,한식,돼지고기 구이/찜,한식 육류 요리 전문점,율현동
50615,멕시카나세곡점,기타 간이,치킨,치킨 전문점,율현동
70750,토프레소,비알코올,카페,커피 전문점,율현동
78533,문,비알코올,카페,커피 전문점,율현동
92979,바란초커피,비알코올,카페,김밥 및 기타 간이 음식점업,율현동
138105,회덮밥잘하는집,한식,기타 한식 음식점,일식 음식점업,율현동
148533,카페이아,비알코올,카페,커피 전문점,율현동
185119,현현,한식,백반/한정식,한식 일반 음식점업,율현동


In [18]:
chromedriver = "chromedriver_win32/chromedriver"
driver = webdriver.Chrome(chromedriver) 

df = data#.sample(200, random_state=42)

df['naver_keyword'] = df['dong'] + " " + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naverURL'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i+1, f"/ {df.shape[0]} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, "#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, "#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naverURL'] = 'https://pcmap.place.naver.com/restaurant/'+df['naverURL']+'/review/visitor#'


# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naverURL'].isnull()]
df = df.drop(columns='naver_keyword')
df.to_csv(f'url_{dongs[n]}_200.csv', encoding='utf-8-sig')

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()

C:\Users\user\AppData\Local\Temp\ipykernel_14064\2924293850.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


이번에 찾을 키워드 : 1 / 29 행 율현동 릴리펏수서점
이번에 찾을 키워드 : 2 / 29 행 율현동 그라운드오리진
이번에 찾을 키워드 : 3 / 29 행 율현동 깡미네
이번에 찾을 키워드 : 4 / 29 행 율현동 멕시카나세곡점
이번에 찾을 키워드 : 5 / 29 행 율현동 토프레소
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x002CA813+48355]
	(No symbol) [0x0025C4B1]
	(No symbol) [0x00165358]
	(No symbol) [0x001909A5]
	(No symbol) [0x00190B3B]
	(No symbol) [0x001BE232]
	(No symbol) [0x001AA784]
	(No symbol) [0x001BC922]
	(No symbol) [0x001AA536]
	(No symbol) [0x001882DC]
	(No symbol) [0x001893DD]
	GetHandleVerifier [0x0052AABD+2539405]
	GetHandleVerifier [0x0056A78F+2800735]
	GetHandleVerifier [0x0056456C+2775612]
	GetHandleVerifier [0x003551E0+616112]
	(No symbol) [0x00265F8C]
	(No symbol) [0x00262328]
	(No symbol) [0x0026240B]
	(No symbol) [0x00254FF7]


In [17]:
#df = pd.read_csv(f'url_{dongs[n]}_200.csv', encoding='utf-8-sig')

In [21]:
df = df.loc[:200]
df

,name,cate_1,cate_2,cate_3,dong,naverURL
0,카페미크,비알코올,카페,커피 전문점,개포동,https://pcmap.place.naver.com/restaurant/11650...
1,스마일명품찹쌀꽈배기,기타 간이,그 외 기타 간이 음식점,"빵류, 과자류 및 당류 소매업",개포동,https://pcmap.place.naver.com/restaurant/10167...
2,부타식당,한식,백반/한정식,한식 일반 음식점업,개포동,https://pcmap.place.naver.com/restaurant/33583...
3,구봉산돼지갈비,한식,돼지고기 구이/찜,한식 육류 요리 전문점,개포동,https://pcmap.place.naver.com/restaurant/38521...
4,랄라홀릭,기타 간이,피자,"피자, 햄버거, 샌드위치 및 유사 음식점업",개포동,https://pcmap.place.naver.com/restaurant/11622...
...,...,...,...,...,...,...
196,리애,일식,일식 회/초밥,일식 음식점업,개포동,https://pcmap.place.naver.com/restaurant/36995...
197,혼,한식,횟집,한식 해산물 요리 전문점,개포동,https://pcmap.place.naver.com/restaurant/17488...
198,참치대가,일식,일식 회/초밥,일식 음식점업,개포동,https://pcmap.place.naver.com/restaurant/37196...
199,빅투치킨,주점,요리 주점,기타 주점업,개포동,https://pcmap.place.naver.com/restaurant/19474...


In [13]:
# 더보기 조금만 누르기, 별점 4.0 이하면 없애rl
chromedriver = "chromedriver_win32/chromedriver"
driver = webdriver.Chrome(chromedriver) 

count = 0 #
current = 0 #현재 진행 상황

goal = len(df['name']) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]

df.index = range(len(df))
for i in range(len(df)): 
    
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naverURL'][i]) 
    thisurl = df['naverURL'][i]
    time.sleep(2)
    print('현재 수집중인 식당 : ', df['name'][i])
    rev = [df['name'][i]]
    #리뷰 더보기 버튼 누르기
    
    k = 0
    while k<5: 
        try:
            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div:nth-child(3) > div.lfH3O > a')
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(1)
            driver.execute_script('return document.querySelector("#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div:nth-child(3) > div.lfH3O > a").click()')
            time.sleep(2)
            k += 1
                        
        except NoSuchElementException:
            print("-모든 리뷰 더보기 완료-")
            break    
        
    #식당 평균 별점 수집
    try:
        rating = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em').text
        print('식당 평균 별점 : ', rating)
        rev.append(rating)
    except:
        rev.append(np.nan)
    
    try:
        type = driver.find_element(By.CSS_SELECTOR, "#_title > span.DJJvD").text
        print('식당 유형 : ', type)
        rev.append(type)
    except:
        rev.append(np.nan)
    

     
        
    #리뷰 데이터 스크래핑을 위한 html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    reviews = ''
    try:
        #키워드 리뷰가 아닌 리뷰글 리스트 검색
        review_lists = soup.select('#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div:nth-child(3) > div.place_section_content > ul > li')
        
        print('총 리뷰 수 : ', len(review_lists))
        
        #리뷰 수가 0이 아닌 경우 리뷰 수집
        if len(review_lists) > 0 :
            
            for j, review in enumerate(review_lists[:10]):
                
                try:
                    
                    #내용 더보기가 있는 경우 내용 더보기를 눌러주기
                    try:
                        review.find(' div.ZZ4OK > a > span.rvCSr > svg')
                        more_content = review.select(' div.ZZ4OK > a > span.rvCSr > svg')
                        more_content.click()
                        time.sleep(1)
   
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK > a > span')
                        
                        if len(user_review) > 0:
                             reviews += user_review[0].text
                       
                        time.sleep(1)            
                
                    
                    
                    except:
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK.IwhtZ > a > span')
                        
                        if len(user_review) > 0:
                            reviews += user_review[0].text
                        
                        time.sleep(1) 
                        
                except NoSuchElementException:
                    print('리뷰 텍스트가 인식되지 않음')
                    continue  
                
        else:
            print('리뷰 선택자가 인식되지 않음')
            time.sleep(1)  

        rev.append(reviews)     
        rev_list.append(rev)
       
        


    # 리뷰가 없는 경우        
    except NoSuchElementException: 
        rev.append(reviews)      
        rev_list.append(rev) 
        time.sleep(2)           
        print("리뷰가 존재하지 않음")

    
            
    # 검색한 창 닫고 검색 페이지로 돌아가기    
    # driver.close()
    # driver.switch_to.window(tabs[0])
    print("기본 페이지로 돌아가기")

        
driver.close()

#스크래핑한 데이터를 데이터 프레임으로 만들기  
column = ["name", 'rate', 'type', "review"]
df2 = pd.DataFrame(rev_list, columns=column)
df2

C:\Users\user\AppData\Local\Temp\ipykernel_28088\3564521626.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


진행상황 : 1/46
현재 수집중인 식당 :  을지로골뱅이앤치킨
식당 유형 :  한식
기본 페이지로 돌아가기
진행상황 : 2/46
현재 수집중인 식당 :  가장맛있는족발
식당 유형 :  족발,보쌈
기본 페이지로 돌아가기
진행상황 : 3/46
현재 수집중인 식당 :  티의수채화TEA.CHAE
식당 유형 :  카페,디저트
기본 페이지로 돌아가기
진행상황 : 4/46
현재 수집중인 식당 :  종로빈대떡
식당 유형 :  전,빈대떡
기본 페이지로 돌아가기
진행상황 : 5/46


InvalidArgumentException: Message: invalid argument: missing command parameters
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0028A813+48355]
	(No symbol) [0x0021C4B1]
	(No symbol) [0x00125358]
	(No symbol) [0x00167579]
	(No symbol) [0x00166B3A]
	(No symbol) [0x00105311]
	(No symbol) [0x0010571E]
	(No symbol) [0x00105AFA]
	GetHandleVerifier [0x004EAABD+2539405]
	GetHandleVerifier [0x0052A78F+2800735]
	GetHandleVerifier [0x0052456C+2775612]
	GetHandleVerifier [0x003151E0+616112]
	(No symbol) [0x00225F8C]
	(No symbol) [0x00104F64]
	(No symbol) [0x00104912]
	GetHandleVerifier [0x0055159C+2959980]
	BaseThreadInitThunk [0x76F87D59+25]
	RtlInitializeExceptionChain [0x77DDB74B+107]
	RtlClearBits [0x77DDB6CF+191]
	(No symbol) [0x00000000]


In [14]:
pd.concat([df, df2.drop(columns='name')], axis=1).to_csv(f'review_{dongs[n]}.csv', encoding='utf-8-sig')
# df2 = pd.read_csv('review_100.csv', index_col=0)

In [ ]:
dongs[n]

'수서동'

In [11]:
rev_list = rev_list[:81]

In [10]:
df.iloc[81]

name                                                      미소야
cate_1                                                     일식
cate_2                                           일식 카레/돈가스/덮밥
cate_3                                                일식 음식점업
dong                                                      개포동
naverURL    https://pcmap.place.naver.com/restaurant/11761...
Name: 81, dtype: object

In [12]:
rev_list

[['현대식당', nan, '백반,가정식', ''],
 ['오븐데이',
  '4.38',
  '치킨,닭강정',
  '매장이 청결하고 맥주🍻치킨🍗 👍👍맥주는 괜찮은데 안주가…주로  포장해와서 먹어서 매장 방문은 처음인데 따끈따끈 너무 맛있어요 파닭에 파도 많고\n닭에 살도 많고 치킨맛집이에요사진찍을사이없이 먹어버렸네요\n담에 또 재방문 의사있습니다오손도손 담소나누며 먹기편한곳\n술안먹어도 안주만 먹어도 맛있는곳이에요^^치맥😆우리동네 맛집~~오븐치킨 맛있어요 더 말할거없는 치킨맛집사이드 토핑이 2개였는데 한개로 바껴서 아쉬워요!!'],
 ['강남초장',
  '4.05',
  '생선회',
  '노포분위기라서 좋고 막회가 맛있어요! 갑오징어숙회? 도 맛있었네요 굿굿굿자주가는 맛집인데 오랜만에 와이프하고 한잔했네요. 테리도 같이...ㅎ 짖으면 쫓겨난다고하니깐 조용하네요. 너무 맛있게 잘먹고 왔어요 너무 싱싱해\n서 좋아요친절한 사장님과 직원분들\n알찬음식\n줄 서는 이유가 있더라고요 🥰👍개포시장에서 이곳을 한번도 안가본 사람은 있어도 한번만 가본 사람은 없을듯!!간단히 술한잔하기 좋은 집회 너무 신선하고 맛있어요. 막장 초장도 직접 만드신 건지 너무 맛있고요.맛있어요 재료가 신선하고 분위기가 좋았어요. 왜 이렇게 사람이 많은지 가보니까 알 것 같아요. 두말필요업는곳, 회쌈하고 갑오징에 해물라면으로  꽉 눌러주면 쥑입니다요. 쐬주에 퐁당빠지기ㅋㅋ메뉴가 특이해서 잘 먹긴 했는데 가성비는 딱히 좋진 않음.'],
 ['현대떡방앗간', '3.92', '떡,한과', ''],
 ['메이린',
  nan,
  '카페',
  '저렴한 커피에 반려동물 가능해서 좋아요.늘 친절하신 사장님 🙆\u200d♀️👍주말에도 아침 일찍 열고 커피맛도 좋아요조아요굿정성이 가득하고 눈꽃빙수 등 특별한 메뉴가 있습니다.말 그대로 수제 대추차입니다.\n사장님이 직접 만드셨다 합니다맛있어요.\n음료가 맛있습니다친절해요'],
 ['바지락칼국수',
  '4.57',
  '칼국수,만두',
  '맛있어요 가성비짱 

In [ ]:
df = df.drop(columns=df.columns[0])